# **Imports**

In [4]:
import pandas as pd

# **Reading Data**

In [5]:
df = pd.read_csv('../Data/combined_data.csv')

# **Data Exploration and Analysis**

In [6]:
df.head()

,Latitude,Longitude,Altitude,Distance,Alignment,Distance_Sensors,Proximity,Wheel_Speed,Velocity,Hall_Effect,Emergency_Stop,Temperature,Humidity,Light_Level
0,37.417885,-121.810845,30.628426,0.0000,False,2.693672,True,2.232098,0.669629,True,True,19.520754,44.387676,70.423761
1,37.417885,-121.810844,30.628426,0.0003,True,1.747220,True,3.645406,1.093622,False,True,22.244082,34.777122,983.361907
2,37.417885,-121.810843,30.628426,0.0006,False,4.850866,True,3.764647,1.129394,False,False,15.588799,67.379012,415.466888
3,37.417885,-121.810842,30.628426,0.0009,False,3.869834,True,4.926673,1.478002,True,False,26.796938,50.141186,628.543683
4,37.417885,-121.810841,30.628426,0.0012,True,1.157138,False,0.970340,0.291102,True,False,23.593743,60.236258,456.516795


In [7]:
df.shape

(5000, 14)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Latitude          5000 non-null   float64
 1   Longitude         5000 non-null   float64
 2   Altitude          5000 non-null   float64
 3   Distance          5000 non-null   float64
 4   Alignment         5000 non-null   bool   
 5   Distance_Sensors  5000 non-null   float64
 6   Proximity         5000 non-null   bool   
 7   Wheel_Speed       5000 non-null   float64
 8   Velocity          5000 non-null   float64
 9   Hall_Effect       5000 non-null   bool   
 10  Emergency_Stop    5000 non-null   bool   
 11  Temperature       5000 non-null   float64
 12  Humidity          5000 non-null   float64
 13  Light_Level       5000 non-null   float64
dtypes: bool(4), float64(10)
memory usage: 410.3 KB


In [9]:
df.describe()

,Latitude,Longitude,Altitude,Distance,Distance_Sensors,Wheel_Speed,Velocity,Temperature,Humidity,Light_Level
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,37.216871,-121.594608,33.121664,0.149850,2.768578,2.759561,0.827868,22.390212,49.828816,492.630568
std,0.156148,0.210838,19.936068,0.086611,1.287386,1.307272,0.392182,7.227560,11.421509,289.483068
min,37.032047,-121.838077,1.576215,0.000000,0.500216,0.500655,0.150196,10.002520,30.005919,0.088550
25%,37.110015,-121.810598,30.628426,0.074925,1.672027,1.625583,0.487675,16.123215,40.122191,242.858835
50%,37.135600,-121.543332,34.451210,0.149850,2.771199,2.777605,0.833281,22.242196,49.835146,488.080833
75%,37.388722,-121.514992,34.496359,0.224775,3.864768,3.891410,1.167423,28.711647,59.641795,739.423933
max,37.417885,-121.266089,64.456110,0.299700,4.999616,4.999721,1.499916,34.998772,69.988824,999.952096


In [10]:
df.isna().sum()

Latitude            0
Longitude           0
Altitude            0
Distance            0
Alignment           0
Distance_Sensors    0
Proximity           0
Wheel_Speed         0
Velocity            0
Hall_Effect         0
Emergency_Stop      0
Temperature         0
Humidity            0
Light_Level         0
dtype: int64

In [11]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level'],
      dtype='object')

In [12]:
alignment_conditions = [
    alignment == True for alignment in df["Alignment"]
]

# Adding labels for Chaining Alignment
df["Alignment_Label"] = ["Aligned" if condition else "Not_Aligned" for condition in alignment_conditions]


In [13]:
# Define conditions for safety based on Hall_Effect and Emergency_Stop
safety_conditions = [
    hall_effect == True and emergency_stop == False
    for hall_effect, emergency_stop in zip(
        df["Hall_Effect"],
        df["Emergency_Stop"]
    )
]

# Adding labels for Safety Decision
df["Safety_Label"] = ["Safe" if condition else "Unsafe" for condition in safety_conditions]

In [14]:
# Speed thresholds (in meters per second - m/s)
threshold_high_speed = 1.5  

# Distance thresholds (in meter)
threshold_low_distance = 0.5 

# Temperature thresholds (in degrees Celsius)
threshold_high_temperature = 30 

# Light level thresholds (in Lux)
threshold_low_light = 100


collision_risk_levels = []

for distance, proximity, wheel_speed, temperature, light_level in zip(
    df["Distance"],
    df["Proximity"],
    df["Wheel_Speed"],
    df["Temperature"],
    df["Light_Level"]
):
    if proximity or wheel_speed > threshold_high_speed:
        collision_risk_levels.append(2)
    elif distance < threshold_low_distance:
        collision_risk_levels.append(1)
    elif temperature > threshold_high_temperature or light_level < threshold_low_light:
        collision_risk_levels.append(1)
    else:
        collision_risk_levels.append(0)


# Adding labels for Collision Avoidance
df["Collision_Avoidance_Label"] = ["Low_Risk" if risk_level == 0 else "Medium_Risk" if risk_level == 1 else "High_Risk" for risk_level in collision_risk_levels]

In [15]:
# Define acceptable distance thresholds for chaining 
max_distance = 5.0  
min_distance = 2.0

# Cart wheel speed treshold in meters per second (m/s) for chaining 
high_speed = 1.5

# Define conditions for chaining based on the data
chaining_conditions = [
    alignment and distance > min_distance and distance < max_distance and not proximity and wheel_speed < high_speed and safety_label == "Safe"
    for alignment, distance, proximity, safety_label, wheel_speed in zip(
        df["Alignment"],
        df["Distance"],
        df["Proximity"],
        df["Safety_Label"],
        df['Wheel_Speed']
    )
]

# Adding labels for Chaining/Unchaining Decision
df["Chaining_Unchaining_Label"] = ["Chaining" if condition else "Unchaining" for condition in chaining_conditions]

In [19]:
# Save the updated combined data with labels to a new CSV file
df.to_csv("../Data/final_combined_data.csv", index=False)

print("Combined data with labels saved.")

Combined data with labels saved.


In [17]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level', 'Alignment_Label', 'Safety_Label',
       'Collision_Avoidance_Label', 'Chaining_Unchaining_Label'],
      dtype='object')

In [18]:
df.isna().sum()

Latitude                     0
Longitude                    0
Altitude                     0
Distance                     0
Alignment                    0
Distance_Sensors             0
Proximity                    0
Wheel_Speed                  0
Velocity                     0
Hall_Effect                  0
Emergency_Stop               0
Temperature                  0
Humidity                     0
Light_Level                  0
Alignment_Label              0
Safety_Label                 0
Collision_Avoidance_Label    0
Chaining_Unchaining_Label    0
dtype: int64